## Dockerfile

- Contains how the application will run inside the container. Dockerfile is used in this project with details. Note that the Docker extension in VS Code helps more in writing the file.
- **Base Image**: Load the base image (e.g., `FROM node:14` for Node.js version 14 or `FROM python`). You can use `FROM` and the base image for your project.
  
  ```Dockerfile
  FROM baseImage
  ```

- **Working Directory**: Set the working directory for your application. This will create a directory for the app you want to dockerize.
  
  ```Dockerfile
  WORKDIR /app
  ```

- **Copy Files**: Use the following command to copy the `package.json` file to the working directory `/app`.
  
  ```Dockerfile
  COPY package.json .
  ```

- **Install Dependencies**: Run the command `npm install` to install all packages from `package.json`.

  ```Dockerfile
  RUN npm install
  ```

- **Copy All Files**: Copy the source files to the destination directory. The command `COPY . .` copies all files in the relative path to the working directory `/app`.
  
  ```Dockerfile
  COPY . .
  ```

- **Expose Port**: Document the port the application will use.
  
  ```Dockerfile
  EXPOSE 4000
  ```

- **Default Command**: Set the default command to be used when launching the container.
  
  ```Dockerfile
  CMD [ "npm", "start" ]
  ```

> **Note**: 
> - `RUN`: The command is triggered while building the Docker image.
> - `CMD`: The command is triggered when we launch the created Docker image.

## Docker Commands

- **Build Image**: Create an image from the Dockerfile. It is better to name the image using `-t "imagename"`. The `.` is used to build from the Dockerfile in the relative path.

  ```bash
  docker build -t express-node-app .
  ```

- **List Images**: To list all Docker images, use the following command:
  
  ```bash
  docker image ls
  ```

- **Run Container**: Use the following command to run the container from the image. It is good practice to give the container a name.
  
  ```bash
  docker run --name express-node-app-container -d express-node-app
  ```

  > `-d`: Run the container in detached mode (so the terminal is not taken over by the container).

- **Show Running Containers**: List all running containers with the following command:
  
  ```bash
  docker ps
  ```

- **Stop Container**: Stop the running container using this command:
  
  ```bash
  docker stop express-node-app-container
  ```

- **Remove Container**: Remove the container forcefully using the following command:
  
  ```bash
  docker rm express-node-app-container -f
  ```

- **Port Forwarding**: To forward the host port 4000 to the container port 4000, use this command:
  
  ```bash
  docker run --name express-node-app-container -d -p 4000:4000 express-node-app
  ```

- **Access Container Terminal**: To access the container's terminal, run:
  
  ```bash
  docker exec -it express-node-app-container bash
  ```

  > `-it`: Interactive terminal.  
  > `express-node-app-container`: Name of the container.  
  > The command uses the bash shell.

- **Check Logs**: View the logs of the running container using this command:
  
  ```bash
  docker logs express-node-app-container
  ```

## General Notes

- Docker Hub (https://hub.docker.com) contains all the images you need, and it is open-source, meaning you can also publish your own images, similar to GitHub.

- A Dockerfile describes how the application will run inside the container.

- You can create multiple containers from a single image.

- Docker images consist of layers (e.g., Layer 1 is `node:14`, Layer 2 could be app files, etc.).

- Each container can hold a service, and the same app can run on multiple containers, with each service on its own container.

- **Node.js application note**: Nodemon is used in `devDependencies` to make the dev server listen for changes and reload automatically during development.

## Docker Optimization

- It is possible to create non-root users in your Dockerfile to avoid running everything as root.

- A `.dockerignore` file specifies which files will be ignored during the image build and container runtime.

- **Command Sequence Optimization**:
  
  ```Dockerfile
  COPY package.json .
  RUN npm install
  COPY . .
  ```

  By separating `package.json` from copying all files, Docker can utilize caching mechanisms. This ensures that `npm install` is triggered only when there are changes in `package.json`, thus preventing unnecessary reinstallation of packages when other files change.

- Environment variables are stored in a `.env` file.

- **Optimization Tip**: Limiting which files are copied is an optimization step to reduce build time and storage space.

## Docker Hot Reload

- Any changes in the local directory will not be reflected in the container unless the image is rebuilt and the container is rerun, which is impractical.

- We used volumes with binding (local_dire:container_dir) to apply hot reload

- **Hot reload** allows syncing between local and container files (mirroring the local directory with the container directory).

- Hot reload can be achieved using the `-v` flag (from volumes) and the absolute path of the local directory and container directory.

  ```bash
  docker run --name express-node-app-container -v F:/DevOps/docker/my-express-app:/app -d -p 4000:4000 express-node-app
  ```

> **Disadvantages of hot reload**: Any files created or removed inside the container will automatically be reflected in the local directory.


## Docker Volumes
**Used Types:**
-  **Bind mount:** like hot reload.
- **Anonymous:** used when storing data on hard drive is needed (like if we use our container as database service).  **Bind mount:** 
> **two way binding:** any change in local dir will be reflected on container dir and vice versa, that is not best practice.
 ```bash
  docker run --name express-node-app-container -v F:/DevOps/docker/my-express-app:/app -d -p 4000:4000 express-node-app
  ```
  > **one way binding:** any change in local dir will be reflected on container dir only and not vice versa.
 ```bash
  docker run --name express-node-app-container -v F:/DevOps/docker/my-express-app:/app:ro -d -p 4000:4000 express-node-app
  ```
> Regarding **one way binding** if we deleted important directory like `node_modules` from local will be deleted from container so if we need to maintain these files and dirs we can use anonymous mount `use another -v before these files and dirs in run container command`. **But there is an easy way is to make binding to the files and dirs you want container to listen to their changes.**

**Note:** The container is a process working on machine on specific port. Working on memory but if we want to store data on hard drive so will use concept of volumes and that is the most common use case for volumes. 

## Docker Compose
-  Is a utility comes with docker to help manage docker containers in a good way. It is not included inside docker itself.
- The file of docker-compose is (.yml file).
- Note, we will use this instead of the long command of running container.
-  `docker-compose up` command will create and start containers
- `docker-compose down` command will stop and remove all containers

![Alt text](image.png)

## Environment Variables
Can be achieved using one of the below:
1. Dockerfile
2. Command line
3. docker-compose

**1. Dockerfile**
```bash
FROM  node:14
WORKDIR  /app
COPY  package.json  .
RUN  npm  install
COPY  .  .
ENV  PORT=4000
EXPOSE  $PORT
CMD  [  "npm",  "run",  "start-dev"  ]
```
**2. Command line**

To check all available flags use below command:
```docker run --help ```

To add environment variables using command lines during running the container and then check them from container terminal do the following:
```
docker run --name express-node-app-container -v F:/DevOps/docker/my-express-app:/app:ro --env PORT=4000 --env NODE_ENV=development -d -p 4000:4000 express-node-app
docker exec -it express-node-app-container bash
root@test:/app# printenv
```
If we have several environment variables, instead of typing them all in the command, we can put them in a `.env` file and then use that file in the command.
```
#.env file
PORT=4000
NODE_ENV=development
```
```
 docker run --name express-node-app-container -v F:/DevOps/docker/my-express-app:/app:ro --env-file ./.env 
-d -p 4000:4000 express-node-app
```



**3. docker-compose**

Note: Take care of indentation in ``docker-compose.yml`` file.

Make section in the file for environment variables:

![Alt text](image-1.png)


Also we can use .env file as below:

![Alt text](image-2.png)

## Docker Environments (Dev, Prod, ...)
- we can make different docker-compose files for different environments, (docker-compose.dev.yml, docker-compose.prod.yml, ...)
- in docker-compose production file > don't forget to remove bind mounting as no hot reload needed.
- you can access container terminal then ```printenv``` to check the assigned environment variables.
- to run the container using one of the below file, you should use file name in the command with -f flag ```docker-compose -f docker-compose.prod.yml up -d```
- **important note**: the above command will run the container from the last built image, if we want to create and run the container from the image with all new changes so use ```--build``` like the following:
 ```docker-compose -f docker-compose.yml -f docker-compose.prod.yml up -d --build```
 - don't forget to add any file starts with docker-compose in .dockerignore file, to avoid them in the container while building the image and running the container
 ```
 #.dockerignore
 node_modules/
Dockerfile
.env
docker-compose*
```
 
 ``` 
#docker-compose.dev.yml file 
#version of docker compose
version: "3"
#services are which containers you have.
services:
	node-app: #any name, this will be name of the app in this container
		container_name: express-node-app-container
		build: .  #will reach the Dockerfile of relative path
		volumes: #relative pathes
			- ./src:/app/src:ro  #local path : container path 	: read only
		ports:
			- "4000:4000"
		environment:
			- NODE_ENV=development  #environment variable
		env_file:
			- ./.env
```

```
#docker-compose.prod.yml file 
#version of docker compose
version: "3"
#services are which containers you have.
services:
	node-app: #any name, this will be name of the app in this container
		container_name: express-node-app-container
		build: .  #will reach the Dockerfile of relative path
		ports:
			- "4000:4000"
		environment:
			- NODE_ENV=production  #environment variable
		env_file:
			- ./.env
```
- now we have duplicated commands between the two files, to overcome that we will write common commands on `docker-compose.yml` and the different parts will be written separately in both files like below:
 ```
#docker-compose.yml
#version of docker compose
version: "3"
#services are which containers you have.
services:
	node-app: #any name, this will be name of the app in this container
			container_name: express-node-app-container
			build: .  #will reach the Dockerfile of relative path
			ports:
				- "4000:4000"
			env_file:
				- ./.env
 ```
```
#docker-compose.dev.yml
#version of docker compose
version: "3"
#services are which containers you have.
services:
	node-app: #any name, this will be name of the app in this container
		volumes: #relative pathes
			- ./src:/app/src:ro  #local path : container path : read only
		environment:
			- NODE_ENV=development  #environment variable
```
```
#docker-compose.dev.yml
#version of docker compose
version: "3"
#services are which containers you have.
services:
	node-app: #any name, this will be name of the app in this container
		environment:
		- NODE_ENV=production  #environment variable
 ```
 

## Multi-Stage Dockerfile

**Problem:** Our current setup uses a single Dockerfile for all environments, with separate Docker Compose files per environment. However, this approach risks including unnecessary packages in the production build.

**Solution:**

1.  We could create separate Dockerfiles for each environment, which would work. **However,** this approach would require modifying each Dockerfile whenever we need additional dependencies, leading to redundancy and potential maintenance issues. A more efficient approach is to use a single, multi-stage Dockerfile.
2. Use single Dockerfile **but** in it we have CMD which should differ between development and production environments.
	```CMD  [  "npm",  "run",  "start-dev"  ]```
so we can override it by using ```command``` in docker-compose like the below:

![Alt text](image-3.png)

Also ```RUN``` command inside Dockerfile, shouldn't be the same for all environments so will use `if` as below: 

![Alt text](image-7.png)

**Note:** Dockerfile will get **ARG** from docker-compose file so their code will be like the below: 

![Alt text](image-4.png)

- to build the image and launch the container (will use --build as we cahnged in the Dockerfile): 

    ```docker-compose -f docker-compose.yml -f docker-compose.dev.yml up -d --build```
- then access container's terminal:
```docker exec -it express-node-app-container bash```
- to find out and package starts with nod use this command in the terminal:
```ls -d nod*```
- checking the container logs using the below command:

    ![Alt text](image-5.png)

3. The third and the **best** solution is to make **multi-stage environment**, this means that Dockerfile has multiple stages 

   ![Alt text](image-8.png)

And we need to pass parameter from docker-compose to indicate which environment will be used prod or dev, so will use `target` as the following code: 

![Alt text](image-9.png)